In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/



1.1 Get the list of animes

With the for loop I get the url of all the pages (50 per page except the last one which has less,383 pag). We 'send' a request to the page, to collect the ursl. We read the html and take the href attributes of the tag. The command tqdm is useful because it shows the progress status.

In [ ]:
urls=[]
for i in tqdm(range(0,20000,50)):
  url='https://myanimelist.net/topanime.php?limit='+str(i)
  
  soup= BeautifulSoup(requests.get(url).text,'html.parser')

  for tag in soup.find_all('tr'):
    links=tag.find_all('a')
    for l in links:
      if type(l.get('id'))== str and len(l.contents[0]) >1:
        urls.append(l.get('href'))

100%|██████████| 400/400 [03:09<00:00,  2.11it/s]


In [ ]:
len(urls)

19134

We need to save it as text

In [ ]:
with open('urls.txt', 'w', encoding='utf-8') as f: #open the file con write, chiude da solo in automatico, lo trovo in file sample data
    for line in urls:
        f.write(line)
        f.write('\n')

In [ ]:
#!ls pages/3pag & just a check, this is used to see what there is inside 3pag

1.2 Crawl animes

Create the folders. Put all of them under "pages". Each folder has a name that refers to the number of the page from which the links it contains come from.

In [ ]:
import os
for page in tqdm(range(1, 384)):
    folder = "page"+str(page)
    path = "/content/drive/MyDrive/Anime_folder"+folder
    os.mkdir(path)

  0%|          | 0/383 [00:00<?, ?it/s]


FileExistsError: ignored

In [ ]:
!rm -rf pages
   #to delete folders that are not needed.

Try for the first 150:ok. Try to collect them in group of 3.

In [ ]:
for page in tqdm(range(0, 3)):  # page 1 --> 383
    fomglder = "/content/drive/MyDrive/Anime_folderpage"+str(page+1)
    update_page = 50*page
    for i in range(0,50):   # 1 -> 50
        url = f'{urls[update_page+i]}'
        response = requests.get(url)   
        filename = r""+folder+"/anime_"+str(update_page+i+1)+".html"
        with open(filename,'w', encoding='utf-8') as f:
            f.write(response.text)

100%|██████████| 3/3 [01:42<00:00, 34.12s/it]


In [ ]:
for page in tqdm(range(4, 8)):  # page 1 --> 383
    folder = "/content/drive/MyDrive/Anime_folderpage"+str(page+1)
    update_page = 50*page
    for i in range(0,50):   # 1 -> 50
        url = f'{urls[update_page+i]}'
        response = requests.get(url)   
        filename = r""+folder+"/anime_"+str(update_page+i+1)+".html"
        with open(filename,'w', encoding='utf-8') as f:
            f.write(response.text)

100%|██████████| 4/4 [00:06<00:00,  1.68s/it]



1.3 Parsing downloaded pages.

In [ ]:
'''col = ['animeTitle', 'animeType', 'animeNumEpisode', 'releaseDate', 'endDate', 'animeNumMembers', 'animeScore', \
           'animeUsers', 'animeRank', 'animePopularity', 'animeDescription', 'animeRelated', 'animeCharacters', \
           'animeVoices', 'animeStaff']
           '''

In [ ]:
def AnimeTitle(html):
  with open(html,'r') as f:
    soup= BeautifulSoup(f)
    animeTitle=soup.find("h1", attrs = {"class": "title-name h1_bold_none"}).string
    return(animeTitle)

In [ ]:
AnimeTitle('/content/drive/MyDrive/Anime_folderpage1/anime_10.html')

'Gintama: The Final'

In [ ]:
animeScore('/content/drive/MyDrive/Anime_folderpage1/anime_13.html')

8.97

In [ ]:
def animeScore(html):
  with open(html,'r') as f:
    soup= BeautifulSoup(f)
    for i in range(10):
      score=soup.find_all('div' ,attrs = {"class": "score-label score-"+str(i)})
      for j in score:
        return float(j.text)

In [ ]:
def AnimeRank(html):
  with open(html,'r') as f:
    soup= BeautifulSoup(f)
    rank=int(str(soup.find(class_="numbers ranked").text).split('#')[-1])  
    return(rank)

In [ ]:
AnimeRank('/content/drive/MyDrive/Anime_folderpage1/anime_13.html')

13

In [ ]:

import re

In [ ]:
def animeNumberofepisode(html_string):
  with open(html_string, 'r') as f:
    soup = BeautifulSoup(f, 'html.parser')
    
    A=soup.find(text=re.compile('Episodes:')).parent.parent.text.split()[-1]      
    print(A)

In [ ]:
animeNumberofepisode('/content/drive/MyDrive/Anime_folderpage1/anime_1.html')

64


In [ ]:
def AnimePopularity(html):
  with open(html,'r') as f:
    soup= BeautifulSoup(f)
    rank=int(str(soup.find(class_="numbers popularity").text).split('#')[-1])  
    return(rank)

In [ ]:
AnimePopularity('/content/drive/MyDrive/Anime_folderpage1/anime_13.html')

23

In [ ]:
def animeDescription(html_string): #I think that this function works well.
  with open(html_string, 'r') as f:
    soup = BeautifulSoup(f, 'html.parser')
    A=soup.find(itemprop="description")
    return A.get_text()

In [ ]:
animeDescription('/content/drive/MyDrive/Anime_folderpage1/anime_10.html')

'New Gintama movie.'

In [ ]:
def animeUsers(html_string): #I think that this function works well.
  with open(html_string, 'r') as f:
    soup = BeautifulSoup(f, 'html.parser')
    A=soup.find(itemprop="ratingCount")
    return int(A.get_text())

In [ ]:
animeUsers('/content/drive/MyDrive/Anime_folderpage1/anime_13.html')

1227199

In [ ]:
def animeType(html_string):
  with open(html_string, 'r') as f:
    soup = BeautifulSoup(f, 'html.parser')
    
    A=soup.find(text=re.compile('Type:')).parent.parent.text.split()[-1]      
    print(A)

In [ ]:
animeType('/content/drive/MyDrive/Anime_folderpage1/anime_11.html')

TV


In [ ]:
def fai_date(a: list):
    
    string = ''
    for i in a:
        string += i
        if i != a[-1]:
            string += ' '
    string = string.replace(',', '')
    date = datetime.strptime(string, '%b %d %Y')
    return date

In [ ]:
def animeRelDate(html_string):
  with open(html_string, 'r') as f:
    soup = BeautifulSoup(f, 'html.parser')
    
    A= fai_date(soup.find(text=re.compile('Aired:'), class_="dark_text").parent.text.split()[1:4]  )   
    print(A)

In [ ]:
animeRelDate('/content/drive/MyDrive/Anime_folderpage1/anime_11.html')

NameError: ignored

In [ ]:
def animeEndDate(html_string):
  with open(html_string, 'r') as f:
    soup = BeautifulSoup(f, 'html.parser')
    
    A= fai_date(soup.find(text=re.compile('Aired:'), class_="dark_text").parent.text.split()[5:8]   )  
    print(A)

In [ ]:
animeEndDate('/content/drive/MyDrive/Anime_folderpage1/anime_11.html')

NameError: ignored

In [ ]:
def animeRelated(html_string):  
  animeRelated = []
  with open(html_string, 'r') as f:
    soup = BeautifulSoup(f, 'html.parser')
    related = soup.find_all("table", {"class":"anime_detail_related_anime"})
    for i in related:
      links = i.find_all('a')
      for link in links:        
          animeRelated.append(f'{link.contents[0]}')

  return animeRelated

In [ ]:
animeRelated('/content/drive/MyDrive/Anime_folderpage1/anime_11.html')

['Gintama', 'Gintama°', 'Gintama.: Porori-hen']

In [ ]:
def animeCharacter(html_string):
  personaggi=[]
  with open(html_string, 'r') as f:
    soup = BeautifulSoup(f, 'html.parser')
    
    A= soup.find_all(class_="h3_characters_voice_actors")    
    for a in A:
      personaggi.append(a.text)
    return(personaggi)

In [ ]:
animeCharacter('/content/drive/MyDrive/Anime_folderpage1/anime_11.html')

['Sakata, Gintoki',
 'Kagura',
 'Katsura, Kotarou',
 'Takasugi, Shinsuke',
 'Shimura, Shinpachi',
 'Kamui',
 'Elizabeth',
 'Imai, Nobume',
 'Sadaharu',
 'Sakamoto, Tatsuma']

In [ ]:
def animeVoices(html_string):
  personaggi=[]
  with open(html_string, 'r') as f:
    soup = BeautifulSoup(f, 'html.parser')
    
    A= soup.find_all(class_="va-t ar pl4 pr4")   
    for a in A:
      personaggi.append(a.text)

      
    return(personaggi)

In [ ]:
animeVoices('/content/drive/MyDrive/Anime_folderpage1/anime_11.html')

['\nSugita, Tomokazu\nJapanese\n',
 '\nKugimiya, Rie\nJapanese\n',
 '\nIshida, Akira\nJapanese\n',
 '\nKoyasu, Takehito\nJapanese\n',
 '\nSakaguchi, Daisuke\nJapanese\n',
 '\nHino, Satoshi\nJapanese\n',
 '\nHirano, Aya\nJapanese\n',
 '\nTakahashi, Mikako\nJapanese\n',
 '\nMiki, Shinichiro\nJapanese\n']

In [ ]:
def createstaff(ll):
    """
    puts all relevant text in a list after cleaning it up a little
    """
    l = []
    for i in ll:
        if i['href'].startswith('https://myanimelist.net/people/') and i['href'] not in l:
            if i.text != '\n\n':
                j = str(i.text).replace('\n', '')
                j = re.sub(' +', ' ', j)
            if j[0] == ' ':
                j = j[1:]
            if j[-1] == ' ':
                j = j[:-1]
            l.append(j)
    return l      





def animeStaff(html_string):
   with open(html_string, 'r') as f:
    soup = BeautifulSoup(f, 'html.parser')
    
    A=  createstaff(soup.find_all('a', href=True))     
    print(A)

In [ ]:
animeStaff('/content/drive/MyDrive/Anime_folderpage1/anime_11.html')

['Sugita, Tomokazu', 'Sugita, Tomokazu', 'Kugimiya, Rie', 'Kugimiya, Rie', 'Ishida, Akira', 'Ishida, Akira', 'Koyasu, Takehito', 'Koyasu, Takehito', 'Sakaguchi, Daisuke', 'Sakaguchi, Daisuke', 'Hino, Satoshi', 'Hino, Satoshi', 'Hirano, Aya', 'Hirano, Aya', 'Takahashi, Mikako', 'Takahashi, Mikako', 'Miki, Shinichiro', 'Miki, Shinichiro', 'Miki, Shinichiro', 'Fujita, Youichi', 'Fujita, Youichi', 'Miyawaki, Chizuru', 'Miyawaki, Chizuru', 'Takamatsu, Shinji', 'Takamatsu, Shinji', 'Kobayashi, Katsuyoshi']


In [ ]:
col = ['animeTitle', 'animeType', 'animeNumEpisode', 'releaseDate', 'endDate', 'animeNumMembers', 'animeScore', 'animeUsers', 'animeRank', 'animePopularity', 'animeDescription', 'animeRelated', 'animeCharacters', 'animeVoices', 'animeStaff']

In [ ]:
records=[]
for soup in urls:
  animeTitle=soup.find("h1", attrs = {"class": "title-name h1_bold_none"}).string
  records.append((animeTitle))

TypeError: ignored

In [ ]:
df=pd.DataFrame(
    {
        'animeTitle':[AnimeTitle], 
        'animeType':[animeType], 
        'animeNumEpisode':[], 
        'releaseDate':[], 
        'endDate':[], 
        'animeNumMembers':[], 
        'animeScore':[], 
        'animeUsers':[], 
        'animeRank':[], 
        'animePopularity':[], 
        'animeDescription':[], 
        'animeRelated':[], 
        'animeCharacters':[], 
        'animeVoices':[], 
        'animeStaff':[]
    }
)

In [ ]:
from datamatrix import DataMatrix
dm=DataMatrix(length=len(col))
dm.title=col()
dm.info=

2.1. Conjunctive query
For the first version of the search engine, we narrow our interest on the Synopsis of each anime. It means that you will evaluate queries only with respect to the anime's description.

2.1.1) Create your index!

Create a file named vocabulary, in the format you prefer, that maps each word to an integer (term_id).

In [1]:
from tqdm import tqdm
from natsort import natsorted
import os
import pandas as pd

In [2]:
documents = []

folder=r"./tsv_anime/"
for anime in tqdm(natsorted(os.listdir(folder))):
  df=pd.read_csv(folder+anime, sep="\t")
  documents.append(df["animeDescription"][0])

FileNotFoundError: ignored

In [ ]:
documents[0]

In [ ]:
#core libraries
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import string
import re

#functions.py
import functions

In [ ]:
contractions={
    "ain't": "am not",
    "aren't": "are not",
    "can't": "cannot",
    "can't've": "cannot have",
    "'cause": "bacause",
    "could've": "could have",
    "couldn't": "could not",
    "didn't": "did not",
    "doesn't": "does not",
    /////////////////////
}

In [ ]:
contractions2={
    "min": "minute",
    "sec": "second"
}

In [ ]:
def pre_processinf(documents):
  stop=stopwords.words("english")
  lmtzr=WordNetLemmatizer()
  remove=["Written", "MAL", "Rewrite"]+["'s"]+[""]
  ps=PorterStemmer()

  #removing contraction+normalization
  document_tmp=fuctions.replace_words(documents.lower(), contractions)
  document_tmp=" ".join(re.split(r"([0-9]+)([a-z]+)", document_tmp))
  document_tmp=functions.replace_words(document_tmp, contractions2)
  document_tmp=re.sub(r"[{}\-''""]".format(string.punctuation)," ",document_tmp)
  document_tmp=word_tokenize(document_tmp)
  document_tmp=[word for word in document_tmp if word not in stop]
  document_tmp=[re.sub(r"[^a-zA-Z0-9]","", word).strip() for word in document_tmp]
  document_tmp=[word for word in document_tmp if word not in remove]
  document_tmp=[lmtzr.lemmatizer(word) for word in document_tmp]
  document_tmp=[ps.stem(word) for word in document_tmp]

  return document_tmp 

In [ ]:
#cleaning the documents
documents_clean=[]
for d in documents:
  documents_clean.append(pre_processing(d))

In [ ]:
documents_clean[0][:10]

In [ ]:
#creating vocabulary
#core libraries
import itertools
import umpy as np

In [ ]:
#the list of all unique words
word_list-list(set(list(itertools.chain.form_iterable(documents_clean))))

In [ ]:
vocabulary=dict(zip(word_list, range(len(word_list))))

In [ ]:
len(vocabulary)

In [ ]:
#view the vocabulary
count=0
for key, mapped_int in vocabulary.items():
  count+=1
  print(key,"-->", mapped_int)
  if count==10: break
  

In [ ]:
import json
file=open("vocabulary.json", "w", encoding="utf-8")
json.dump(vocabulary, file)
file.close()

In [ ]:
#import the saved vocabularry
with open("vocabulary.json") as f:
  vocabulary=json.load(f)

In [ ]:
#2.1.CONJUCTIVE QUERY

In [ ]:
def word_to_int(document, vocabulary):
  int_doc=np.zeros(len(document), dtype=np.int64)
  for i, word in enumerate(document):
    int_doc[i]=vocabulary[word]

  return np.sort(int_doc)

In [ ]:
documents_mapped=[]
for d in documents_clean:
  documents_mapped.append(word_to_int(d, vocabulary))

In [ ]:
documents_mapped[0]

In [ ]:
from collections import defaultdict

In [ ]:
inverted_index=defaultdict(list)

In [ ]:
for i,d in enumerate(documents_mapped):
  for word in set(d):
    Inverted_index[str(word)].append(i)

In [ ]:
count=0
for key, lis in Inverted_index.items():
  count+=1
  print("word: ", key,"-->","documents: ", lis)
  if count==1: break

In [ ]:
import json

file=open("Inverted_index_v1.json","w", encoding="utf-8")
json.dump(Inverted_index, file)
file.close()

In [ ]:
with open("Inverted_index_v1.json" ) as f:
  Inverted_index=json.load(f)

In [ ]:
def search_engine_v1(query_text):
  query_clean=pre_processing(query_text)
  query_int=word_to_int(query_clean, vocabulary)

  index=[]
  for query in query_int:
    index.append(set(Inverted_index[str(query)]))
  index=list(index[0].intersection(*index))

  with open("./anime_url.txt", "r", encoding="utf-8") as f:
    lines=f.readlines()

  anime_path=[]
  url=[]

  for idx in index:
    name="/anime_"+str(idx+1)+".tsv"
    anime_path.append(name)
    url.append(lines[idx])
  
  animes_df=[]
  folder=r"./tsv_anime/"
  cols=["animeTitle", "animeDescription"]
  for i,anime_tsv in enumerate(anime_path):
    df=pd.read_csv(folder+anime_tsv, sep="\t", usecols=cols)
    df["animeURL"]=url[i]
    animes_df.append(df)
  
  frame=pd.concat(animes_df, ignore_index=True)
  display(frame)


In [ ]:
#input query
query_text=input("Insert the query: ")

search_engine_v1(query_text)

### 2.2

In [ ]:
n=len(documents)
n_words=len(vocabulary)

In [ ]:
def word_to_int2(document, vocabulary):
  int_doc=np.zeros(len(vocabulary), dtype=np.int64)

  for word in document:
    int_doc[vocabulary[word]]+=1
  return int_doc

In [ ]:
tf=[]
for d in documents_clean:
  tf.append(word_to_int2(d, vocabulary))

In [ ]:
tf[0][tf[0]>0]

In [ ]:
nj=np.zeros(n_words, dtype=np.float64)

for d in documents_mapped:
  for word in set(d):
    nj[word]+=1

In [ ]:
nj

In [ ]:
idf=np.zeros(n_words, dtype=np.float64)
idf=np.log(n/nj)/np.log(n)

In [ ]:
tfIdf=np.multiply(tf, idf)

In [ ]:
np.shape(tfIdf)

In [ ]:
from collections import defaultdict

In [ ]:
Inverted_indexv2=defaultdict(list)

In [ ]:
for i,d in enumerate(documents_mapped):
  for word in set(d):
    Inverted_indexv2[str(word)].append((i,tfIdf[i, int(word)]))

In [ ]:
count=0
for key, lis in Inverted_indexv2.items():
  count+=1
  print(key, "-->",lis)
  if count==1: break
  

In [ ]:
import json

file=open("inverted_index_v2.json", "w", encoding="utf-8")
json.dump(Inverted_indexv2, file)
file.close()

In [ ]:
with open("Inverted_index_v2.json") as f:
  Inverted_indexv2 = json.load(f)

In [ ]:
import heapq 

In [ ]:
def search_engine2(query, k, tfIdf):
  query_clean=pre_processing(query)
  query_int=word_to_int2(query_clean, vocabulary)

  match_list=[]
  lenMatch=[]
  max_lenMatch=(0,-1)
  for i,query in enumerate(np.where(query_int>0)[0]):
    lis=Inverted_indexv2[str(query)]
    match_list.append(lis)
    tmplis, tmplen = i, len(lis)
    if tmplen>max_lenMatch[1]:
      max_lenMatch=(tmplis, tmplen)
    lenMatch.append(len(lis))

  enough, match_list=functions.intersection_all(k, match_list)
  m=len(query_clean)
  if enough:
    scores=functions.scoresK(match_list, tfIdf, m, query_int)
    topscore, topk=fucntions.find_topK(k, scores)
  else:
    scores=functions.scoreALL(match_list, tfIdf, m, lenMatch)
    k=len(scores)
    topscore, topk=functions.find_topK(k, scores)
  with open("./anime_url.txt", "r", encoding="utf-8") as f:
    lines=f.readlines()

  anime_path=[]
  url=[]

  for idx in topk:
    name = "/anime_"+str(idx+1)+".tsv"
    anime_path.append(name)
    url.append(lines[idx])

  animes_df=[]
  folder=r"./tsv_anime/"
  cols=["animeTitle", "animeDescription"]
  for i, anime_tsv in enumerate(anime_path):
    df=pd.read_csv(folder+anime_tsv, sep="\t", usecols=cols)

    df["animeURL"]=url[i]
    df["animeScores"]=topscore[i]
    animes_df.append(df)

  frame=pd.concat(animes_df, ignore_index=True)
  display(frame)
  return


### Input and searching

In [ ]:
query=input("Insert the query: ")
k=int(input("Insert k: "))

search_engine2(query, k, tfIdf)